In [0]:
%pip install databricks-vectorsearch

In [0]:
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient


CATALOG = 'manufacturing_dev'
SCHEMA = 'work_agent_barney'


client = VectorSearchClient()

In [0]:
index = client.create_endpoint_and_wait(
    name="master_sensory_data_endpoint",
    endpoint_type="STANDARD"
)

In [0]:
index = client.create_delta_sync_index_and_wait(
    endpoint_name="master_sensory_data_endpoint",
    index_name=f"{CATALOG}.{SCHEMA}.master_sensory_panel_joined_index",
    primary_key="id",
    source_table_name=f"{CATALOG}.{SCHEMA}.master_sensory_panel_joined_silver",
    pipeline_type="TRIGGERED",
    embedding_dimension=1536,
    embedding_vector_column="data_embedding",
    verbose=True,
)

In [0]:
index = client.create_delta_sync_index_and_wait(
    endpoint_name="master_sensory_data_endpoint",
    index_name=f"{CATALOG}.{SCHEMA}.master_sensory_responses_collected_index",
    primary_key="id",
    source_table_name=f"{CATALOG}.{SCHEMA}.master_sensory_responses_collected_silver",
    pipeline_type="TRIGGERED",
    embedding_dimension=1536,
    embedding_vector_column="data_embedding",
    verbose=True,
)